# Trabalho 2 - Previsão de Resultados de Futebol

Código: webscraping dos dados necessários das partidas através do Site FBref

<strong>Torneios de Scraping</strong>
<li>
   Brasileirão Série A
</li>

## Instalação e Configuração

In [ ]:
import time
import undetected_chromedriver as uc

try:
    driver = uc.Chrome(headless=False) 
    print("Driver iniciado com sucesso.")

except Exception as e:
    print(f"Erro ao iniciar o driver: {e}")
    exit()

url = "https://fbref.com/en/comps/24/2025/schedule/2025-Serie-A-Scores-and-Fixtures"
driver.get(url)

: 

## Obtenção das Partidas e Dados

In [2]:
from selenium.webdriver.common.by import By

table_div = driver.find_elements(By.CLASS_NAME, 'table_container')
rows = table_div[0].find_elements(By.TAG_NAME, 'tr')
links = []
matches = []

for row in rows:
  try:
    td_elements = row.find_elements(By.TAG_NAME, 'td')
    th_element = row.find_element(By.TAG_NAME, 'th')
    link_td = td_elements[-2].find_elements(By.TAG_NAME, 'a')
    notes_td = td_elements[-1]
    
    if notes_td.text == 'Match Postponed':
      continue
    if link_td[0].text == 'Head-to-Head':
      break

    aux = []
    aux.append(th_element.text)
    aux.extend([element.text for element in td_elements])

    matches.append(aux)
    links.append(link_td[0].get_attribute("href"))
  except:
    continue

In [3]:
matches

[['1',
  'Sat',
  '2025-03-29',
  '18:30',
  'São Paulo',
  '2.4',
  '0–0',
  '0.4',
  'Sport Recife',
  '37,856',
  'Estádio do Morumbi',
  'Felipe Fernandes de Lima',
  'Match Report',
  ''],
 ['1',
  'Sat',
  '2025-03-29',
  '18:30',
  'Juventude',
  '0.7',
  '2–0',
  '0.6',
  'Vitória',
  '6,547',
  'Estádio Alfredo Jaconi',
  'Paulo Cesar Zanovelli da Silva',
  'Match Report',
  ''],
 ['1',
  'Sat',
  '2025-03-29',
  '18:30',
  'Fortaleza',
  '0.9',
  '2–0',
  '0.9',
  'Fluminense',
  '19,484',
  'Estádio Castelão',
  'Rodrigo José Pereira de Lima',
  'Match Report',
  ''],
 ['1',
  'Sat',
  '2025-03-29',
  '18:30',
  'Grêmio',
  '0.8',
  '2–1',
  '2.9',
  'Atlético Mineiro',
  '20,111',
  'Arena do Grêmio',
  'Raphael Claus',
  'Match Report',
  ''],
 ['1',
  'Sat',
  '2025-03-29',
  '18:30',
  'Cruzeiro',
  '1.2',
  '2–1',
  '1.7',
  'Mirassol',
  '40,837',
  'Estádio Governador Magalhães Pinto',
  'Alex Gomes Stefano',
  'Match Report',
  ''],
 ['1',
  'Sun',
  '2025-03-30',
  

In [6]:
import concurrent.futures
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import threading

driver_init_lock = threading.Lock()


# Cada thread executa a função para um link
def scrape_match_data(link):
    """
    Inicia um novo driver, abre um link, extrai os dados da partida
    e fecha o driver.
    """
    driver_thread = None
    try:
        with driver_init_lock:
            driver_thread = uc.Chrome(headless=False)
        driver_thread.get(link)

        team_stats_base = driver_thread.find_elements(By.ID, 'team_stats')
        team_stats_extra = driver_thread.find_elements(By.ID, 'team_stats_extra')

        # Loop de tentativa para garantir que a página carregou
        retry_count = 0
        while len(team_stats_base) == 0 and retry_count < 10:
            time.sleep(5 + retry_count * 2)
            driver_thread.get(link)
            team_stats_base = driver_thread.find_elements(By.ID, 'team_stats')
            team_stats_extra = driver_thread.find_elements(By.ID, 'team_stats_extra')
            retry_count += 1
        
        if len(team_stats_base) == 0:
            print(f"Falha ao carregar dados de: {link}")
            return None
        
        # Base
        rows = team_stats_base[0].find_elements(By.TAG_NAME, 'tr')
        rows = rows[1:9]

        stat_base = []
        values = []
        aux = []
        for i in range(8):
            if i % 2 == 1:
                teams_td = rows[i].find_elements(By.TAG_NAME, 'td')
                for j in range(2):
                    team_td = teams_td[j]
                    div = team_td.find_element(By.TAG_NAME, "div")
                    parts = div.text.split(" ")
                    if len(parts) == 1:
                        aux.append(parts[0])
                    elif j % 2 == 0:
                        aux.append(parts[0])
                        aux.append(parts[2])
                    else:
                        aux.append(parts[2])
                        aux.append(parts[4])
            else:
                tag_stat_name = rows[i].find_element(By.TAG_NAME, 'th')
                stat_base.append(tag_stat_name.text)
        values.append(aux)

        # Extra
        rows = team_stats_extra[0].find_elements(By.TAG_NAME, 'div')
        rows = rows[1:]
        aux = []

        first_part = [row.text for row in rows[3:15]]
        second_part = [row.text for row in rows[18:31]]
        third_part = [row.text for row in rows[34:47]]

        aux.append(first_part[0])
        aux.append(first_part[2])
        aux.append(first_part[3])
        aux.append(first_part[5])
        aux.append(first_part[6])
        aux.append(first_part[8])
        aux.append(first_part[9])
        aux.append(first_part[11])

        aux.append(second_part[1])
        aux.append(second_part[3])
        aux.append(second_part[4])
        aux.append(second_part[6])
        aux.append(second_part[7])
        aux.append(second_part[9])
        aux.append(second_part[10])
        aux.append(second_part[12])

        aux.append(third_part[1])
        aux.append(third_part[3])
        aux.append(third_part[4])
        aux.append(third_part[6])
        aux.append(third_part[7])
        aux.append(third_part[9])
        aux.append(third_part[10])
        aux.append(third_part[12])

        values.append(aux)

        return values

    except Exception as e:
        print(f"Erro ao processar {link}: {e}")
        return None
    
    finally:
        if driver_thread:
            driver_thread.quit()

# --- Gerenciamento da Pool de Threads ---

print("Driver principal (para links) fechado.")
try:
    driver.quit() 
except:
    pass 

MAX_WORKERS = 4

deeper_infos = []

print(f"Iniciando scraping de {len(links)} partidas com {MAX_WORKERS} workers paralelos...")

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # O 'executor.map' executa a função e retorna os resultados
    # NA MESMA ORDEM em que os links foram enviados.
    results = list(executor.map(scrape_match_data, links))

deeper_infos = results
print("Scraping paralelo concluído.")

Driver principal (para links) fechado.
Iniciando scraping de 327 partidas com 4 workers paralelos...
Scraping paralelo concluído.


In [7]:
deeper_infos[0][0]

['64%',
 '36%',
 '495',
 '581',
 '251',
 '328',
 '3',
 '15',
 '1',
 '8',
 '1',
 '1',
 '3',
 '3']

## Conversão para CSV

In [8]:
# Registro do CSV => Todas as infos de uma partida

columns = ['Week', 'Day', 'Date', 'Time', 'Home', 'xG_H', 'Score', 'xG_A', 'Away', 'Attendance', 'Venue', 'Referee', 'Possession_H', 'Possession_A',
           'PassCompleted_H', 'PassTotal_H', 'PassCompleted_A', 'PassTotal_A', 'ShotsTarget_H', 'ShotsTotal_H', 'ShotsTarget_A', 'ShotsTotal_A',
           'SaveCompleted_H', 'SaveTotal_H', 'SaveCompleted_A', 'SaveTotal_A', 'Fouls_H', 'Fouls_A', 'Corners_H', 'Corners_A', 'Crosses_H',
           'Crosses_A', 'Touches_H', 'Touches_A', 'Tackles_H', 'Tackles_A', 'Interceptions_H', 'Interceptions_A', 'Aerials_H', 'Aerial_A',
           'Clearances_H', 'Clearances_A', 'Offsides_H', 'Offsides_A', 'Goal_Kicks_H', 'Goal_Kicks_A', 'Throw_Ins_H', 'Throw_Ins_A',
           'Long_Balls_H', 'Long_Balls_A']

In [9]:
print(len(columns))

50


In [10]:
merged_matches = []

for i in range(len(matches)):
    match_details = deeper_infos[i] 
    
    if match_details is not None:
        match_row = matches[i].copy()
        match_row.extend(match_details[0])
        match_row.extend(match_details[1])
        
        merged_matches.append(match_row)
    else:
        print(f"Aviso: Pulando partida {i} ({matches[i][3]} vs {matches[i][7]}) devido a falha no scraping.")

In [11]:
merged_matches[0]

['1',
 'Sat',
 '2025-03-29',
 '18:30',
 'São Paulo',
 '2.4',
 '0–0',
 '0.4',
 'Sport Recife',
 '37,856',
 'Estádio do Morumbi',
 'Felipe Fernandes de Lima',
 'Match Report',
 '',
 '64%',
 '36%',
 '495',
 '581',
 '251',
 '328',
 '3',
 '15',
 '1',
 '8',
 '1',
 '1',
 '3',
 '3',
 '17',
 '21',
 '6',
 '2',
 '20',
 '2',
 '676',
 '443',
 '16',
 '13',
 '6',
 '5',
 '9',
 '18',
 '10',
 '30',
 '1',
 '2',
 '6',
 '17',
 '28',
 '17',
 '62',
 '62']

In [12]:
for i in range(len(merged_matches)):
    merged_matches[i] = merged_matches[i][0:12] + merged_matches[i][14:]

In [13]:
len(merged_matches[0])

50

In [14]:
import csv

csv_filename = 'matches_2025.csv'

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(columns)
    csv_writer.writerows(merged_matches)